# read Fmcc from .json

In [1]:
import json
import codecs
import numpy as np


In [2]:
def ImportMfccData(pathMfccSave):
    objDataRead = codecs.open(pathMfccSave, 'r', encoding='utf-8').read()
    Rdata = json.loads(objDataRead)
    
    RX = np.array(Rdata['X'])
    RY = np.array(Rdata['Y'])
    Rmapping = Rdata['mapping']
    return RX,RY,Rmapping

In [6]:
pathMfccSave ='fmcc_marine_mamal.json'
RX,RY,Rmapping = ImportMfccData(pathMfccSave)
print('data importing completed')

data importing completed


In [7]:
np.unique(RY, return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8]),
 array([  16,  213,   57,    3,  264,   71,  201, 1622,   72]))

# prepare data

In [8]:
from sklearn.model_selection import train_test_split
test_size = 0.25
validation_size = 0.2
def prepareData (X, Y, test_size, validation_size):
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size)
    X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=validation_size)
        
    X_train = X_train[..., np.newaxis] 
    X_validation = X_validation[..., np.newaxis] 
    X_test = X_test[..., np.newaxis] 

    return X_train, X_validation, X_test, y_train, y_validation, y_test


In [9]:

X_train, X_validation, X_test,y_train, y_validation, y_test = prepareData (RX, RY, test_size, validation_size)

In [10]:
## debugging
X_train.shape

(1511, 40, 100, 1)

# Training model

In [11]:

import tensorflow as tf
import tensorflow.keras as keras
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd 

In [18]:
def Build_model(input_shape):
    
    # Instantiate model
    model = keras.Sequential()
    
    # 1st block
    model.add(keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(keras.layers.MaxPool2D((3, 3), strides=(2,2), padding='same'))
    model.add(keras.layers.BatchNormalization()) # normalizes the activation at the layer, speeds up training
              
    # 2nd block
    model.add(keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=input_shape))
    model.add(keras.layers.MaxPool2D((3, 3), strides=(2,2), padding='same'))
    model.add(keras.layers.BatchNormalization())
                  
              
    # 3rd block
    model.add(keras.layers.Conv2D(128, (2, 2), activation='relu', input_shape=input_shape))
    model.add(keras.layers.MaxPool2D((2, 2), strides=(2,2), padding='same'))
    model.add(keras.layers.BatchNormalization())
              
    # flatten
    model.add(keras.layers.Flatten()) # flatten  output
    model.add(keras.layers.Dense(128, activation='relu'))
    model.add(keras.layers.Dropout(0.3)) # randomly drops neurons
            
    # output layer 
    model.add(keras.layers.Dense(9, activation='softmax')) 
    
    print(model.summary())
    
    return model

end


In [14]:
# build the cnn net
input_shape = (X_train.shape[1], X_train.shape[2], X_train.shape[3])
model = Build_model(input_shape)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 38, 98, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 19, 49, 32)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 19, 49, 32)        128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 17, 47, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 9, 24, 64)         0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 9, 24, 64)         256       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 23, 128)        3

2022-04-05 11:30:24.363762: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-04-05 11:30:24.364075: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-05 11:30:24.365813: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [19]:
# compile the network
optimizer = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, 
              loss="sparse_categorical_crossentropy",
              metrics=['accuracy'])

In [16]:
# train the cnn 
history = model.fit(X_train, y_train, validation_data=(X_validation, y_validation), batch_size=24, epochs=20)


2022-04-05 11:30:34.717488: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-04-05 11:30:34.734989: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2494180000 Hz


Epoch 1/20
63/63 [==============================] - 14s 211ms/step - loss: 0.9206 - accuracy: 0.7319 - val_loss: 0.8706 - val_accuracy: 0.8889
Epoch 2/20
63/63 [==============================] - 13s 207ms/step - loss: 0.2245 - accuracy: 0.9188 - val_loss: 0.3106 - val_accuracy: 0.9286
Epoch 3/20
63/63 [==============================] - 12s 191ms/step - loss: 0.1458 - accuracy: 0.9433 - val_loss: 0.1816 - val_accuracy: 0.9365
Epoch 4/20
63/63 [==============================] - 12s 197ms/step - loss: 0.1100 - accuracy: 0.9519 - val_loss: 0.1527 - val_accuracy: 0.9180
Epoch 5/20
63/63 [==============================] - 10s 159ms/step - loss: 0.1128 - accuracy: 0.9389 - val_loss: 0.1241 - val_accuracy: 0.9392
Epoch 6/20
63/63 [==============================] - 9s 144ms/step - loss: 0.1029 - accuracy: 0.9400 - val_loss: 0.1187 - val_accuracy: 0.9180
Epoch 7/20
63/63 [==============================] - 10s 157ms/step - loss: 0.1034 - accuracy: 0.9411 - val_loss: 0.1144 - val_accuracy: 0.9233


In [17]:
# evaluate 

test_error, test_accuracy = model.evaluate(X_test, y_test, verbose=1)
print("Accuracy on test set is: {}".format(test_accuracy))

20/20 [==============================] - 1s 70ms/step - loss: 0.1304 - accuracy: 0.9381
Accuracy on test set is: 0.938095211982727


# Predict Sample

In [72]:
def getKey_fromValue (dictionary, value):
    list_Key=  list(dictionary.keys())
    list_Value=  list(dictionary.values())
    position = list_Value.index(value)
    return list_Key[position]

In [76]:
def Predict(model, X, Rmapping):
    X = X[np.newaxis, ...] # 4D np arrya
    
    prediction = model.predict(X) 
    
    # get the index 
    predict_index = np.argmax(prediction, axis=1) 
    
    return getKey_fromValue(Rmapping,predict_index[0])
    

In [77]:
index_test=10
Predict(model,X_test[index_test],Rmapping)

'Bearded Seal'